In [ ]:
using CairoMakie
using Arya
using LilGuys
using CSV, DataFrames

In [ ]:
using HDF5

In [ ]:
function read_traj(name)
    local positions, velocities, times
    
    h5open("$name/trajectory.hdf5", "r") do f
        positions = f["positions"][:, :, :]
        velocities = f["velocities"][:, :, :]
        times = -f["times"][:]
    end

    return positions, velocities, times
end

In [ ]:
function read_distribution(name)
    return LilGuys.read_fits(joinpath(name, "peris_apos.fits"))
end

In [ ]:
function compare_peris(families)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "pericentre / kpc",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.pericenter, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
function compare_apos(families)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "apocentre / kpc",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.apocenter, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
using Printf

In [ ]:
using StatsBase

In [ ]:
function compare_stats(families)

    for (label, df) in families
        @printf "%16s%12.2f [%0.2f %0.2f] [[%0.2f %0.2f]]\n" label median(df.pericenter) quantile(df.pericenter, 0.16) quantile(df.pericenter, 0.84) quantile(df.pericenter, 0.001) quantile(df.pericenter, 0.999)
    end
end

In [ ]:
families = [
    "strict" => read_distribution("reported_uncertanties"),
    "+gaia sys" => read_distribution("systematic_errors"),
    "+solar lsr sys" => read_distribution("sys_and_solar"),
    #"all studies" => read_distribution("all_studies"),
    "iorio" => read_distribution("iorio")
    ]

In [ ]:
compare_stats(families)

In [ ]:
compare_peris(families)

In [ ]:
compare_apos(families)

In [ ]:
families = [
    "+gaia sys" => read_distribution("systematic_errors"),
    "V+21" => read_distribution("vasiliev_nolmc"),
    "+LMC" => read_distribution("vasiliev_lmc"),
    ]

In [ ]:
compare_stats(families)

In [ ]:
compare_peris(families)

## Trajectories

In [ ]:
traj = read_traj("vasiliev_lmc")

In [ ]:
traj_no = read_traj("vasiliev_nolmc")

In [ ]:
V_T2GYR = 0.97779

In [ ]:
reshape

In [ ]:
scatter(lmc_traj.x, lmc_traj.y)

In [ ]:
scatter(pos[1, 1, :], pos[2, 1, :])

In [ ]:
# loads in trajectory of lmc in Vasiliev 2021
lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/trajlmc.txt"
lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])

lmc_x = LilGuys.lerp(lmc_traj.time, lmc_traj.x)
lmc_y = LilGuys.lerp(lmc_traj.time, lmc_traj.y)
lmc_z = LilGuys.lerp(lmc_traj.time, lmc_traj.z)
lmc_v_x = LilGuys.lerp(lmc_traj.time, lmc_traj.v_x)
lmc_v_y = LilGuys.lerp(lmc_traj.time, lmc_traj.v_y)
lmc_v_z = LilGuys.lerp(lmc_traj.time, lmc_traj.v_z)

times_v = traj[3] * T2GYR / V_T2GYR
pos = reshape([lmc_x.(times_v) lmc_y.(times_v) lmc_z.(times_v)]', (3, 1, :))
vel = reshape([lmc_v_x.(times_v) lmc_v_y.(times_v) lmc_v_z.(times_v)]', (3, 1, :))

In [ ]:
traj_lmc = pos, vel, traj[3]
traj_scl_lmc = pos .- traj[1], vel .- traj[2], traj[3]


In [ ]:
function plot_r_t_traj!(traj; alpha=0.01, color=:black, kwargs...)
    positions, velocities, times = traj
    for i in 1:size(positions, 2)
        x = times * T2GYR
        y = calc_r(positions[:, i, :])
        lines!(x, y; alpha=alpha, color=color, kwargs...)
    
    end
end

In [ ]:
function plot_x_y_traj!(traj; alpha=0.01, color=:black, kwargs...)
    positions, velocities, times = traj
    for i in 1:size(positions, 2)
        x = positions[2, i, :]
        y = positions[3, i, :]
        
        lines!(x, y; alpha=alpha, color=color, kwargs...)
    
    end
end

In [ ]:
COLORS[1]

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="time / Gyr", ylabel = "radius / kpc",
    xgridvisible=false, ygridvisible=false
)

plot_r_t_traj!(traj, label="+LMC", color=COLORS[1])
plot_r_t_traj!(traj_no, label="V+21 potential")

axislegend(unique=true)
fig

In [ ]:
mkpath("figures")
figdir = "./figures/"

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="time / Gyr", ylabel = "radius / kpc",
    xgridvisible=false, ygridvisible=false
)

plot_r_t_traj!(traj, label="Scl-MW", color=COLORS[1])
plot_r_t_traj!(traj_scl_lmc, label="Scl-LMC", color=COLORS[2])
plot_r_t_traj!(traj_lmc, label="LMC-MW", color=:black, alpha=1, linewidth=3, )


axislegend(unique=true)

Makie.save("$figdir/scl_lmc_mw_r_t_samples.pdf", fig)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="y / kpc", ylabel="z / kpc",
    xgridvisible=false, ygridvisible=false, 
    aspect=DataAspect(),
)
plot_x_y_traj!(traj_scl_lmc, label="+LMC", color=:black)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="y / kpc", ylabel="z / kpc",
    xgridvisible=false, ygridvisible=false, 
    aspect=DataAspect(),
)
plot_x_y_traj!(traj, label="+LMC", color=COLORS[1])
plot_x_y_traj!(traj_no, label="V+21 potential")
plot_x_y_traj!(traj_lmc, label="LMC", alpha=1, color=COLORS[2], linewidth=3)

axislegend(unique=true)

fig